In [24]:
import uproot
import pandas as pd
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import fastjet
import pdg

pdg_api = pdg.connect()

In [10]:
def load_events(filepath):
    with uproot.open(filepath) as f:
        # Tree handling
        tree_key = next((k for k in f.keys() if "events" in k.lower()), None)
        if not tree_key:
            raise ValueError("No tree found matching 'events' in file.")

        #Branch handling
        tree = f[tree_key]
        branches = tree.keys()
        
        def match(branches, target):
            return next((b for b in branches if b.lower() == target.lower()), None)

        keys_needed = ["id", "px", "py", "pz", "e"]
        matched_keys = {k: match(branches, k) for k in keys_needed}

        if any(v is None for v in matched_keys.values()):
            missing = [k for k, v in matched_keys.items() if v is None]
            raise KeyError(f"Missing branches: {missing}")

        # Sample the data to check for nested lists
        sample = tree.arrays(list(matched_keys.values()), entry_stop=5, library="ak")
        is_nested = any(isinstance(sample[k][0], (list, ak.Array)) for k in matched_keys)

        if is_nested:
            print("⚠️ Nested tree detected — flattening.")
            full = tree.arrays(list(matched_keys.values()), library="ak")
            flat = ak.flatten(ak.zip({k: full[matched_keys[k]] for k in matched_keys}), axis=1)
            return ak.to_dataframe(flat).reset_index(drop=True)
        else:
            print("✅ Flat tree detected — using pandas.")
            return tree.arrays(list(matched_keys.values()), library="pd")


dps_data = load_events("dps/ttbb_v2.root")      
sps_data = load_events("sps/sps_ttbb_showered.root")

✅ Flat tree detected — using pandas.
⚠️ Nested tree detected — flattening.


In [34]:
counts_sps = df_sps["id"].value_counts()
counts_dps = df_dps["id"].value_counts()

# Map using API
def label_with_name(pdgid):
    try:
        name = pdg_api.get_particle_by_mcid(pdgid).name
        return f"{pdgid} ({name})"
    except Exception:
        return f"{pdgid} (Unknown)"

counts_sps = counts_sps.rename(index=label_with_name)
counts_dps = counts_dps.rename(index=label_with_name)

print(counts_sps)


id
22 (gamma)         2784143
211 (pi+)          1147728
-211 (pi-)         1140066
321 (K+)            140255
-321 (K-)           138737
130 (K(L)0)         136184
2212 (p)             87995
2112 (n)             84833
-2212 (pbar)         77069
-2112 (nbar)         75775
-11 (e+)             23957
11 (e-)              23938
13 (mu-)              7140
-14 (nubar_mu)        7009
-13 (mu+)             6999
12 (nu_e)             6910
-12 (nubar_e)         6891
14 (nu_mu)            6868
16 (nu_tau)           4106
-16 (nubar_tau)       4106
Name: count, dtype: int64


In [35]:
print(counts_dps)

id
22 (gamma)         2488865
211 (pi+)          1025094
-211 (pi-)         1018809
321 (K+)            125580
-321 (K-)           123578
130 (K(L)0)         122280
2212 (p)             78950
2112 (n)             75750
-2112 (nbar)         67363
-2212 (pbar)         67337
11 (e-)              21880
-11 (e+)             21879
-12 (nubar_e)         6692
12 (nu_e)             6691
-13 (mu+)             6673
13 (mu-)              6572
14 (nu_mu)            6546
-14 (nubar_mu)        6445
16 (nu_tau)           3886
-16 (nubar_tau)       3886
Name: count, dtype: int64


In [13]:
df_dps = dps_data.copy()
df_sps = sps_data.copy()
# --- Step 2: Compute derived quantities
def compute_pt_eta_phi(df):
    df["pt"] = np.sqrt(df["px"]**2 + df["py"]**2)
    df["eta"] = 0.5 * np.log((df["e"] + df["pz"]) / (df["e"] - df["pz"] + 1e-6))
    df["phi"] = np.arctan2(df["py"], df["px"])
    return df

df_dps = compute_pt_eta_phi(df_dps)
df_sps = compute_pt_eta_phi(df_sps)

# Particle-level selection
def is_b_hadron(pdgid):
    return abs(pdgid)%1000//100 == 5 or abs(pdgid)%10000//1000 == 5

def is_bb_hadron(pdgid):
    return (abs(pdgid)%1000//100 == 5 and abs(pdgid)%100//10 ==5) or (abs(pdgid)%10000//1000 == 5 and abs(pdgid)%1000//100 == 5)

def count_particles(df, label):
    print(f"\n[{label}]")
    print("Total particles:", len(df))
    print("Charged leptons:", len(df[np.abs(df["id"]).isin([11, 13])]))
    print("Neutrinos:", len(df[np.abs(df["id"]).isin([12, 14, 16])]))
    print("b-hadrons:", len(df[df["id"].apply(is_b_hadron)]))
    print("bb-hadrons:", len(df[df["id"].apply(is_bb_hadron)]))

count_particles(df_dps, "DPS")
count_particles(df_sps, "SPS")


/Users/hunterrr/playground/UZH/.venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/hunterrr/playground/UZH/.venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)



[DPS]
Total particles: 5284756
Charged leptons: 57004
Neutrinos: 34146
b-hadrons: 0
bb-hadrons: 0

[SPS]
Total particles: 5910709
Charged leptons: 62034
Neutrinos: 35890
b-hadrons: 0
bb-hadrons: 0
